In [1]:
import requests
import pandas as pd
import ftplib
import io
import re
import json
import datetime

try:
    from requests_html import HTMLSession
except Exception:
    print("""Warning - Certain functionality 
             requires requests_html, which is not installed.
             
             Install using: 
             pip install requests_html
             
             After installation, you may have to restart your Python session.""")

    
base_url = "https://query1.finance.yahoo.com/v8/finance/chart/"


C:\Users\srema\AppData\Local\Continuum\anaconda3\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [14]:
def force_float(elt):
    
    try:
        return float(elt)
    except:
        return elt

def _convert_to_numeric(s):

    if "M" in s:
        s = s.strip("M")
        return force_float(s) * 1_000_000
    
    if "B" in s:
        s = s.strip("B")
        return force_float(s) * 1_000_000_000
    
    return force_float(s)

In [17]:
def get_top_crypto():
    
    '''Gets the top 100 Cryptocurrencies by Market Cap'''      

    session = HTMLSession()
    
    resp = session.get("https://finance.yahoo.com/cryptocurrencies?offset=0&count=100")
    
    tables = pd.read_html(resp.html.raw_html)               
                    
    df = tables[0].copy()

    
    df["% Change"] = df["% Change"].map(lambda x: float(str(x).strip("%").\
                                                               strip("+").\
                                                               replace(",", "")))
#     del df["52 Week Range"]
#     del df["1 Day Chart"]
    
    fields_to_change = [x for x in df.columns.tolist() if "Volume" in x \
                        or x == "Market Cap" or x == "Circulating Supply"]
    
    for field in fields_to_change:
        
        if type(df[field][0]) == str:
            df[field] = df[field].map(lambda x: _convert_to_numeric(str(x)))
            
            
    session.close()        
                
    return df

In [18]:
get_top_crypto()

,Symbol,Name,Price (Intraday),Change,% Change,Market Cap,Volume in Currency (Since 0:00 UTC),Volume in Currency (24Hr),Total Volume All Currencies (24Hr),Circulating Supply,52 Week Range,1 Day Chart
0,BTC-USD,Bitcoin USD,47034.470000,-1539.98,-3.17,8.891130e+11,3.156800e+10,3.156800e+10,3.156800e+10,1.8903e+07,NaN,NaN
1,ETH-USD,Ethereum USD,3907.410000,-180.09,-4.41,4.641140e+11,2.145400e+10,2.145400e+10,2.145400e+10,1.18778e+08,NaN,NaN
2,BNB-USD,BinanceCoin USD,528.190000,-14.76,-2.72,8.810200e+10,1.795000e+09,1.795000e+09,1.795000e+09,1.66801e+08,NaN,NaN
3,USDT-USD,Tether USD,1.000600,-0.0005,-0.05,7.636000e+10,6.752800e+10,6.752800e+10,6.752800e+10,7.6311e+10,NaN,NaN
4,SOL1-USD,Solana USD,179.320000,-5.19,-2.81,5.521800e+10,2.849000e+09,2.849000e+09,2.849000e+09,3.07925e+08,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
95,SUSHI-USD,Sushi USD,5.612800,-0.4946,-8.10,7.142000e+08,2.036520e+08,2.036520e+08,2.036520e+08,1.27244e+08,NaN,NaN
96,GNO-USD,Gnosis USD,447.880000,-17.43,-3.75,6.738800e+08,6.563000e+06,6.563000e+06,6.563000e+06,1.505e+06,NaN,NaN
97,ZRX-USD,0x USD,0.771351,-0.037883,-4.68,6.537170e+08,4.747600e+07,4.747600e+07,4.747600e+07,8.47496e+08,NaN,NaN
98,XWC-USD,WhiteCoin USD,0.841696,+0.031706,3.91,6.263910e+08,1.057600e+07,1.057600e+07,1.057600e+07,7.44201e+08,NaN,NaN


In [19]:
session = HTMLSession()

resp = session.get("https://coinmarketcap.com/")

tables = pd.read_html(resp.html.raw_html) 

In [20]:
tables

[    Unnamed: 0    #                 Name       Price  24h %   7d %  \
 0          NaN  1.0       Bitcoin1BTCBuy  $46,999.17  3.02%  1.47%   
 1          NaN  2.0      Ethereum2ETHBuy   $3,898.05  4.30%  2.26%   
 2          NaN  3.0  Binance Coin3BNBBuy     $526.78  2.77%  6.34%   
 3          NaN  4.0       Tether4USDTBuy       $1.00  0.09%  0.00%   
 4          NaN  5.0        Solana5SOLBuy     $179.17  2.49%  4.11%   
 ..         ...  ...                  ...         ...    ...    ...   
 95         NaN  NaN       Immutable XIMX       $4.68    NaN    NaN   
 96         NaN  NaN            DecredDCR      $64.31    NaN    NaN   
 97         NaN  NaN     Voyager TokenVGX       $3.12    NaN    NaN   
 98         NaN  NaN       OMG NetworkOMG       $6.09    NaN    NaN   
 99         NaN  NaN             AnkrANKR       $0.11    NaN    NaN   
 
                   Market Cap                         Volume(24h)  \
 0   $888.45B$888,445,589,536          $31,475,304,952669,699 BTC   
 1   $46

In [146]:
session = HTMLSession()

resp = session.get("https://web.archive.org/web/20210110154437/https://coinmarketcap.com/")

# resp.html.raw_html

In [145]:
session = HTMLSession()

resp = session.get("https://coinmarketcap.com/")

# resp.html.raw_html

In [28]:
resp.full_text

AttributeError: 'HTMLResponse' object has no attribute 'full_text'

In [2]:
session = HTMLSession()

resp = session.get("https://web.archive.org/web/20161204115058/https://coinmarketcap.com")

session.close() 
resp.html.raw_html.decode("utf-8")  

'<!DOCTYPE html>\n<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->\n<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8"> <![endif]-->\n<!--[if IE 8]>         <html class="no-js lt-ie9"> <![endif]-->\n<!--[if gt IE 8]><!--> <html class="no-js"> <!--<![endif]-->\n    <head><script src="//archive.org/includes/analytics.js?v=cf34f82" type="text/javascript"></script>\n<script type="text/javascript">window.addEventListener(\'DOMContentLoaded\',function(){var v=archive_analytics.values;v.service=\'wb\';v.server_name=\'wwwb-app28.us.archive.org\';v.server_ms=692;archive_analytics.send_pageview({});});</script>\n<script type="text/javascript" src="/_static/js/bundle-playback.js?v=aToEIw17" charset="utf-8"></script>\n<script type="text/javascript" src="/_static/js/wombat.js?v=UHAOicsW" charset="utf-8"></script>\n<script type="text/javascript">\n  __wm.init("https://web.archive.org/web");\n  __wm.wombat("https://coinmarketcap.com/","20161204115058","https://web.a

In [31]:
test = resp.html.raw_html.decode("utf-8")  

In [2]:
from io import StringIO
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.text = StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [3]:
import time
import codecs

def get_html_text(url):
    
    session = HTMLSession()
    resp = session.get(url)
    text = resp.html.raw_html.decode("utf-8")  
    session.close()   
    
    return text

def parse_html(url):
    
    text = get_html_text(url)
    text_split = text.split('\n')
    test_str = strip_tags(text_split[-21])
    raw_data = test_str[test_str.find('{"props"'):]
    
    return raw_data
    
def save_json(path, filename, text):
    
    text_file = codecs.open(f"{path}/{filename}.json", "w", "utf-8")
    n = text_file.write(text)
    text_file.close()
    
def get_filename(url):
    year_month = url.split('/')
    return year_month[4][:6]
    
def scrape_json(path, url_list):
    
    for url in url_list:
        time.sleep(10)
        raw_data = parse_html(url)
        filename = get_filename(url)
        save_json(path, filename, raw_data)

In [177]:
url_list = [
    "https://web.archive.org/web/20211201014034/https://coinmarketcap.com",
    "https://web.archive.org/web/20211101030300/https://coinmarketcap.com",
    "https://web.archive.org/web/20211001123833/https://coinmarketcap.com",
    "https://web.archive.org/web/20210901013215/https://coinmarketcap.com",
    "https://web.archive.org/web/20210801024525/https://coinmarketcap.com",
    "https://web.archive.org/web/20210701002735/https://coinmarketcap.com",
]

path = "data/json"

scrape_json(path, url_list)

In [35]:
url = "https://web.archive.org/web/20200301050706/https://coinmarketcap.com"
test_url = url.split('/')
raw_date = test_url[4][:8]
f"{raw_date[:4]}-{raw_date[4:6]}-{raw_date[6:]}"

'2020-11-15'

In [46]:
url_list = [
    "https://web.archive.org/web/20211201014034/https://coinmarketcap.com",
    "https://web.archive.org/web/20211101030300/https://coinmarketcap.com",
    "https://web.archive.org/web/20211001123833/https://coinmarketcap.com",
    "https://web.archive.org/web/20210901013215/https://coinmarketcap.com",
    "https://web.archive.org/web/20210801024525/https://coinmarketcap.com",
    "https://web.archive.org/web/20210701002735/https://coinmarketcap.com",
]

indices = [i for i, s in enumerate(url_list) if '20210901013215' in s][0]

url_list[indices]


'https://web.archive.org/web/20210901013215/https://coinmarketcap.com'

In [40]:
import pandas as pd

def get_date(url):
    split_url = url.split('/')
    raw_date = split_url[4][:8]
    return f"{raw_date[:4]}-{raw_date[4:6]}-{raw_date[6:]}"
    
def list_to_dataframe(raw_data):
    
    columns = ['rank', "Name", "Market_cap", "USD_pric", 
               "Available_supply", "symbol", "USD_volume_24h", "USD_percent_change_24h"]
    
    df = pd.DataFrame.from_records(raw_data, columns=columns)
    df['Ticker'] = df['symbol'] + '-USD'
    df['Date'] = get_date(url)
    df['Date'] = pd.to_datetime(df['Date'])
    
    return df

def open_csv(path):
    

    with open(path, newline='') as f:
        reader = csv.reader(f)
        url_list = list(reader)[1:]
        
    return [url[0] for url in url_list]

def find_url(path, filename):
    
    url_list = open_csv(path)
    index = [i for i, s in enumerate(url_list) if filename in s][0]
    
    return url_list[index]
    
def old_site(path, filename):
    
    url = find_url(path, filename)
    
    stripped_text = strip_tags(text).split('\n')
    begin_index = stripped_text.index('                    % Change (24h)Price Graph (7d)')
    end_index = stripped_text.index('                        Next 100  →')
    indexed_text = stripped_text[begin_index:end_index] 

    spaceless_text = [item.replace(" ", "") for item in indexed_text]
    nostar_text = [item.replace("*", "") for item in spaceless_text]
    not_empty_text = [item for item in nostar_text if len(item)>0]

    raw_data = []
    for i in range(1, len(not_empty_text), 8):
        raw_data.append(not_empty_text[i:i+8])
    
    df = list_to_dataframe(raw_data)

    return df

In [41]:
url = "https://web.archive.org/web/20161204115058/https://coinmarketcap.com"
old_site(url)

,rank,Name,Market_cap,USD_pric,Available_supply,symbol,USD_volume_24h,USD_percent_change_24h,Ticker,Date
0,1,Bitcoin,"$12,345,019,923",$770.43,"16,023,462",BTC,"$50,288,000",0.28%,BTC-USD,2016-12-04
1,2,Ethereum,"$668,146,993",$7.72,"86,592,180",ETH,"$8,100,460",-1.43%,ETH-USD,2016-12-04
2,3,Ripple,"$235,603,528",$0.006547,"35,987,750,899",XRP,"$934,478",-1.63%,XRP-USD,2016-12-04
3,4,Litecoin,"$190,874,520",$3.92,"48,728,404",LTC,"$1,488,790",-0.17%,LTC-USD,2016-12-04
4,5,Monero,"$112,905,240",$8.38,"13,475,446",XMR,"$1,465,530",0.92%,XMR-USD,2016-12-04
...,...,...,...,...,...,...,...,...,...,...
95,96,VeriCoin,"$738,897",$0.024755,"29,847,926",VRC,"$2,890",0.20%,VRC-USD,2016-12-04
96,97,Blocknet,"$726,066",$0.185670,"3,910,516",BLOCK,$925,-5.15%,BLOCK-USD,2016-12-04
97,98,Aeon,"$723,694",$0.055430,"13,056,047",AEON,"$1,004",19.45%,AEON-USD,2016-12-04
98,99,WorldCoin,"$666,966",$0.006026,"110,673,906",WDC,$97,15.10%,WDC-USD,2016-12-04


In [26]:
x = 'B'
int(x)

ValueError: invalid literal for int() with base 10: 'B'

[2, 3, 4, 5]

In [61]:
import numpy as np

# Scraping and parsing the old site

def get_date(url):
    ''' Return date from URL'''
    
    split_url = url.split('/')
    raw_date = split_url[4][:8]
    return f"{raw_date[:4]}-{raw_date[4:6]}-{raw_date[6:]}"
    
def list_to_dataframe(raw_data, url):
    ''' Convert list of lists with scraped data to a dataframe and add some extra columns'''
    
    columns = ['rank', "Name", "Market_cap", "USD_pric", 
               "Available_supply", "symbol", "USD_volume_24h", "USD_percent_change_24h"]
    
    df = pd.DataFrame.from_records(raw_data, columns=columns)
    df['Ticker'] = df['symbol'] + '-USD'
    df['Date'] = get_date(url)
    df['Date'] = pd.to_datetime(df['Date'])
    
    return df

def find_url(path, filename):
    ''' Find the URL of the file that could not be parsed by looking for a substring in the URL list'''
    
    url_list = open_csv(path)
    date = filename.replace(".json", "")
    index = [i for i, s in enumerate(url_list) if date in s][0]
    
    return url_list[index]

def text_cleaner(text):
    ''' Remove duplicates from text'''
    
    clean_text = text

    for i in range(len(text)):
        is_int = False
        try:
            int(text[i])
            is_int = True
        except:
            pass
        
        if is_int:
            unique_item = text[i+1]
            if unique_item == text[i+6]:
                del text[i+6]
    
    return clean_text

def parse_old_site(url):
    ''' Scrape and parse the old site and return a pandas dataframe
    
        This funtion is triggered if parsing the JSON file from the "scrape_json" could not be parsed.
        The function first scrapes the intended page, then removes all the HTML so that 
        only a list with the data remains. This list is then index and written to a dataframe.
        
        params:
            url_path: string - Directory of the URL list
            filename: string - Name of the JSON file that could not be parsed
        
        returns:
            df: pandas dataframe - dataframe with scraped data            
    '''
    
#     url = "https://web.archive.org/web/20171002080443/https://coinmarketcap.com"
    text = get_html_text(url)
    stripped_text = strip_tags(text).split('\n')
    spaceless_text = [item.replace(" ", "") for item in stripped_text]
    nostar_text = [item.replace("*", "") for item in spaceless_text]
    not_empty_text = [item for item in nostar_text if len(item)>0]
    begin_index = not_empty_text.index('%Change(24h)PriceGraph(7d)')

    end_index = not_empty_text.index('NotMineable')

    indexed_text = not_empty_text[begin_index:end_index-2] 

    clean_text = text_cleaner(indexed_text)

    raw_data = []
    for i in range(1, len(clean_text), 8):
        raw_data.append(clean_text[i:i+8])
    
    df = list_to_dataframe(raw_data, url)

    return df

In [59]:


# parse_old_site()

,rank,Name,Market_cap,USD_pric,Available_supply,symbol,USD_volume_24h,USD_percent_change_24h,Ticker,Date
0,1,BTC,Bitcoin,"$73,181,769,228",$4408.65,"16,599,587","$1,527,580,000",2.40%,"16,599,587-USD",2017-10-02
1,2,ETH,Ethereum,"$28,211,662,765",$297.21,"94,920,370","$349,425,000",-0.77%,"94,920,370-USD",2017-10-02
2,3,XRP,Ripple,"$7,821,760,306",$0.203990,"38,343,841,883","$90,088,500",3.58%,"38,343,841,883-USD",2017-10-02
3,4,BCH,BitcoinCash,"$6,915,925,861",$415.39,"16,649,075","$227,461,000",-1.74%,"16,649,075-USD",2017-10-02
4,5,LTC,Litecoin,"$2,867,423,522",$53.90,"53,198,557","$96,614,300",-0.23%,"53,198,557-USD",2017-10-02
...,...,...,...,...,...,...,...,...,...,...
95,96,CFI,Cofound.it,"$42,783,000",$0.131640,"325,000,000","$285,671",-2.83%,"325,000,000-USD",2017-10-02
96,97,SNM,SONM,"$40,925,611",$0.123508,"331,360,000","$2,289,380",35.23%,"331,360,000-USD",2017-10-02
97,98,MLN,Melon,"$40,904,614",$68.24,"599,400","$462,677",-0.74%,"599,400-USD",2017-10-02
98,99,VTC,Vertcoin,"$40,392,832",$1.01,"39,965,600","$627,923",1.34%,"39,965,600-USD",2017-10-02


In [63]:
import csv

def open_csv(path):
    ''' Open a CSV and return a list with the contents'''

    with open(path, newline='') as f:
        reader = csv.reader(f)
        url_list = list(reader)[1:]
        
    return [url[0] for url in url_list]

url_list = open_csv('data/market_cap/urls.csv')

for url in url_list:
    try:
        parse_old_site(url)
        print('SUCCESS:', url)
    except Exception as e:
        print(url)
        print(e)

https://web.archive.org/web/20211201014034/https://coinmarketcap.com
'%Change(24h)PriceGraph(7d)' is not in list
https://web.archive.org/web/20211101030300/https://coinmarketcap.com
'%Change(24h)PriceGraph(7d)' is not in list
https://web.archive.org/web/20211001123833/https://coinmarketcap.com
'%Change(24h)PriceGraph(7d)' is not in list
https://web.archive.org/web/20210901013215/https://coinmarketcap.com
'%Change(24h)PriceGraph(7d)' is not in list
https://web.archive.org/web/20210801024525/https://coinmarketcap.com
'%Change(24h)PriceGraph(7d)' is not in list
https://web.archive.org/web/20210701002735/https://coinmarketcap.com
'%Change(24h)PriceGraph(7d)' is not in list
https://web.archive.org/web/20210601012136/https://coinmarketcap.com
'%Change(24h)PriceGraph(7d)' is not in list
https://web.archive.org/web/20210501003024/https://coinmarketcap.com
'%Change(24h)PriceGraph(7d)' is not in list
https://web.archive.org/web/20210401013444/https://coinmarketcap.com
'%Change(24h)PriceGraph(7d)

In [13]:
def text_cleaner2(text):
    ''' Remove duplicates from text'''
    
#     clean_text = text

    for i in range(len(text)):
        is_int = False
#         print(clean_text[1])
#         print(int(clean_text[1]))
        try:
#             print(clean_text[i])
            int(clean_text[i])
            is_int = True
#             print(clean_text[i], is_int)
        except:
            pass
        
        if is_int:
            unique_item = text[i+1]
            if unique_item == text[i+6]:
                del text[i+6]
#                 print()
    
    return text

In [ ]:
def text_cleaner3(text):
    ''' Remove duplicates from text'''
    
    clean_text = []

    for i in range(len(text)):
        is_int = False

        try:
            int(clean_text[i])
            is_int = True
        except:
            pass

In [7]:
url = "https://web.archive.org/web/20200301050706/https://coinmarketcap.com"

text = get_html_text(url)
stripped_text = strip_tags(text).split('\n')
spaceless_text = [item.replace(" ", "") for item in stripped_text]
nostar_text = [item.replace("*", "") for item in spaceless_text]
not_empty_text = [item for item in nostar_text if len(item)>0]

not_empty_text
# try:
#     begin_index = not_empty_text.index('%Change(24h)PriceGraph(7d)')
# except:
#     begin_index = not_empty_text.index('PriceGraph(7d)')
# # begin_index = not_empty_text.index('%Change(24h)PriceGraph(7d)')

# end_index = not_empty_text.index('NotMineable')

# indexed_text = not_empty_text[begin_index:end_index-2] 

# # clean_text = text_cleaner2(indexed_text)

# # print('is same:', clean_text == indexed_text)

# for i in range(1, len(indexed_text), 9):
#     print(indexed_text[i:i+9])
#     print(clean_text[i:i+9])
    
    
# deduplicated_text = [i for n, i in enumerate(indexed_text) if i not in indexed_text[:n]]
# deduplicated_text
# indexed_text


# Symbol staat er twee keer in. Mogelijk aparte functie voor site vanaf 2017? 

["window.addEventListener('DOMContentLoaded',function(){varv=archive_analytics.values;v.service='wb';v.server_name='wwwb-app209.us.archive.org';v.server_ms=321;archive_analytics.send_pageview({});});",
 '__wm.init("https://web.archive.org/web");',
 '__wm.wombat("https://coinmarketcap.com/","20200301050706","https://web.archive.org/","web","/_static/",',
 '\t"1583039226");',
 "!function(){if('PerformanceLongTaskTiming'inwindow){varg=window.__tti={e:[]};",
 'g.o=newPerformanceObserver(function(l){g.e=g.e.concat(l.getEntries())});',
 "g.o.observe({entryTypes:['longtask']})}}();",
 'window.dataLayer=window.dataLayer||[];',
 'window.dataLayer.push({"appType":"nextjs"});',
 "(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':newDate().getTime(),event:'gtm.js'});varf=d.getElementsByTagName(s)[0],j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src='//web.archive.org/web/20200301050706/https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);})(wi

In [5]:
url = "https://web.archive.org/web/20200301050706/https://coinmarketcap.com"

text = get_html_text(url)
stripped_text = strip_tags(text).split('\n')
begin_index = stripped_text.index('                    % Change (24h)Price Graph (7d)')
end_index = stripped_text.index('                        Next 100  →')
indexed_text = stripped_text[begin_index:end_index] 

spaceless_text = [item.replace(" ", "") for item in indexed_text]
nostar_text = [item.replace("*", "") for item in spaceless_text]
not_empty_text = [item for item in nostar_text if len(item)>0]

not_empty_text
for i in range(1, len(not_empty_text), 8):
    print(not_empty_text[i:i+8])



# stripped_text[-200:]
# text_split = text.split('\n')
# text_split
# test_str = strip_tags(text_split[-21])
# raw_data = test_str[test_str.find('{"props"'):]
# raw_data

ValueError: '                    % Change (24h)Price Graph (7d)' is not in list

In [27]:
not_empty_text

['%Change(24h)PriceGraph(7d)',
 '1',
 'Bitcoin',
 '$12,345,019,923',
 '$770.43',
 '16,023,462',
 'BTC',
 '$50,288,000',
 '0.28%',
 '2',
 'Ethereum',
 '$668,146,993',
 '$7.72',
 '86,592,180',
 'ETH',
 '$8,100,460',
 '-1.43%',
 '3',
 'Ripple',
 '$235,603,528',
 '$0.006547',
 '35,987,750,899',
 'XRP',
 '*',
 '$934,478',
 '-1.63%',
 '4',
 'Litecoin',
 '$190,874,520',
 '$3.92',
 '48,728,404',
 'LTC',
 '$1,488,790',
 '-0.17%',
 '5',
 'Monero',
 '$112,905,240',
 '$8.38',
 '13,475,446',
 'XMR',
 '$1,465,530',
 '0.92%',
 '6',
 'EthereumClassic',
 '$67,364,750',
 '$0.778677',
 '86,511,801',
 'ETC',
 '$385,835',
 '0.44%',
 '7',
 'Dash',
 '$60,596,271',
 '$8.75',
 '6,924,821',
 'DASH',
 '$869,609',
 '0.06%',
 '8',
 'Steem',
 '$44,055,707',
 '$0.194741',
 '226,227,177',
 'STEEM',
 '$151,809',
 '11.87%',
 '9',
 'Augur',
 '$36,276,460',
 '$3.30',
 '11,000,000',
 'REP',
 '*',
 '$64,283',
 '1.27%',
 '10',
 'NEM',
 '$31,029,030',
 '$0.003448',
 '8,999,999,999',
 'XEM',
 '*',
 '$24,007',
 '-1.28%',
 '11'

In [45]:

session = HTMLSession()

resp = session.get("https://web.archive.org/web/20210110154437/https://coinmarketcap.com/")

session.close() 

test = resp.html.raw_html.decode("utf-8")  
text = test.split('\n')

test_str = strip_tags(text[-21]) # checken of dit altijd zo is !!!

data = test_str[test_str.find('{"props"'):]

In [185]:
import os
import collections

def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)
    

def parse_json(file_path):
    
    row_values = []
     
    col_names = []
    
    with open(directory) as json_file:
        json_data = json.load(json_file)
    
    json_dict = json_data['props']['initialState']['cryptocurrency']['listingLatest']['data']

    for i in range(len(json_dict)):
        flat_dict = flatten(json_dict[i], parent_key='', sep='_')
        item_list = list(flat_dict.values())
        
        row_values.append(item_list)
        
        if len(col_names) == 0:
            col_names = list(flat_dict.keys())
    
    df = pd.DataFrame(row_values, columns=col_names)
    df.columns = df.columns.str.strip('quote_')
    
    return df


def clean_data(df):
    
    df_clean = df.copy()
    df_clean['Ticker'] = df_clean['symbol'] + '-USD'
    df_clean['Date'] = pd.to_datetime(df_clean['last_updated']).dt.date
    df_clean = df_clean.rename(columns={"id": "ID", "nam": "Name", "USD_market_cap": "Market_cap"})
    
    return df_clean


def save_df(path):

    for file in os.listdir(path):
        filename = file.replace(".json", ".csv")
        
        df = parse_json(file_path)
        df_clean = clean_data(df)
        df_clean.to_csv(f"data/market_cap/{filename}", index=False)
        
        

In [186]:
path = "data/json"

save_df(path)

In [183]:
pd.to_datetime(1490195805433502912, unit='SS')

ValueError: cannot cast unit SS

In [52]:
data = test_str[test_str.find('{"props"'):]

In [58]:
def save_json(path, filename, text):
    
    text_file = open(f"{path}/{filename}.json", "w")
    n = text_file.write(text)
    text_file.close()

In [120]:
path = 'data/json'
filename = 'test'
save_json(path, filename, data)

In [141]:
file_path = f"{path}/{filename}.json"

parse_json(file_path)

,id,nam,symbol,slug,max_supply,circulating_supply,al_supply,last_updated,USD_pric,USD_volume_24h,USD_percent_change_1h,USD_percent_change_24h,USD_percent_change_7d,USD_market_cap,USD_last_updated,rank,noLazyLoad
0,1,Bitcoin,BTC-USD,bitcoin,2.100000e+07,1.859641e+07,1.859641e+07,2021-01-10T15:41:03.000Z,39790.654176,6.573528e+10,-0.422332,-3.393856,17.790660,7.399632e+11,2021-01-10T15:41:03.000Z,1,True
1,1027,Ethereum,ETH-USD,ethereum,NaN,1.141948e+08,1.141948e+08,2021-01-10T15:41:02.000Z,1304.654419,3.771341e+10,-0.905235,5.824488,41.918916,1.489848e+11,2021-01-10T15:41:02.000Z,2,True
2,825,Tether,USDT-USD,tether,NaN,2.422431e+10,2.478163e+10,2021-01-10T15:41:08.000Z,1.000397,1.196793e+11,-0.052278,-0.082858,-0.089081,2.423392e+10,2021-01-10T15:41:08.000Z,3,True
3,52,XRP,XRP-USD,xrp,1.000000e+11,4.540403e+10,9.999083e+10,2021-01-10T15:42:02.000Z,0.330189,8.531281e+09,-2.153986,0.626029,48.012324,1.499189e+10,2021-01-10T15:42:02.000Z,4,True
4,2,Litecoin,LTC-USD,litecoin,8.400000e+07,6.624562e+07,6.624562e+07,2021-01-10T15:42:02.000Z,174.014448,1.232930e+10,-1.559849,1.258082,13.546431,1.152769e+10,2021-01-10T15:42:02.000Z,5,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1659,Gnosis,GNO-USD,gnosis-gno,1.000000e+07,1.504589e+06,1.000000e+07,2021-01-10T15:42:06.000Z,121.020937,8.588960e+05,-1.340051,9.009099,47.929240,1.820868e+08,2021-01-10T15:42:06.000Z,96,None
96,1214,Lisk,LSK-USD,lisk,NaN,1.268929e+08,1.429022e+08,2021-01-10T15:42:03.000Z,1.435357,2.396700e+07,-1.176783,-1.148334,23.987117,1.821366e+08,2021-01-10T15:42:03.000Z,97,None
97,3662,HedgeTrade,HEDG-USD,hedgetrade,NaN,3.392315e+08,1.000000e+09,2021-01-10T15:42:05.000Z,0.516040,1.394323e+05,-0.612473,-4.495825,15.803934,1.750570e+08,2021-01-10T15:42:05.000Z,98,None
98,3437,ABBC Coin,ABBC-USD,abbc-coin,1.500000e+09,8.441424e+08,1.500000e+09,2021-01-10T15:42:05.000Z,0.205466,4.830258e+07,-0.625698,-13.264025,-4.418634,1.734423e+08,2021-01-10T15:42:05.000Z,99,None


In [82]:
import json

with open(directory) as json_file:
    json_data = json.load(json_file)
    
json_dict = json_data['props']['initialState']['cryptocurrency']['listingLatest']['data']

In [95]:
json_dict[0]

[{'id': 1,
  'name': 'Bitcoin',
  'symbol': 'BTC',
  'slug': 'bitcoin',
  'max_supply': 21000000,
  'circulating_supply': 18596406,
  'total_supply': 18596406,
  'last_updated': '2021-01-10T15:41:03.000Z',
  'quote': {'USD': {'price': 39790.654176302625,
    'volume_24h': 65735275082.33181,
    'percent_change_1h': -0.42233167,
    'percent_change_24h': -3.39385588,
    'percent_change_7d': 17.79065962,
    'market_cap': 739963160068.1191,
    'last_updated': '2021-01-10T15:41:03.000Z'}},
  'rank': 1,
  'noLazyLoad': True},
 {'id': 1027,
  'name': 'Ethereum',
  'symbol': 'ETH',
  'slug': 'ethereum',
  'max_supply': None,
  'circulating_supply': 114194827.999,
  'total_supply': 114194827.999,
  'last_updated': '2021-01-10T15:41:02.000Z',
  'quote': {'USD': {'price': 1304.6544187981901,
    'volume_24h': 37713409473.90333,
    'percent_change_1h': -0.90523487,
    'percent_change_24h': 5.82448818,
    'percent_change_7d': 41.91891621,
    'market_cap': 148984786952.79465,
    'last_updat

In [98]:
import collections

def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

In [114]:
ff = flatten(json_dict[0], parent_key='', sep='_')
list(ff.values())

[1,
 'Bitcoin',
 'BTC',
 'bitcoin',
 21000000,
 18596406,
 18596406,
 '2021-01-10T15:41:03.000Z',
 39790.654176302625,
 65735275082.33181,
 -0.42233167,
 -3.39385588,
 17.79065962,
 739963160068.1191,
 '2021-01-10T15:41:03.000Z',
 1,
 True]

In [ ]:
ff = flatten(json_dict[0], parent_key='', sep='_')

dinges = []

for key, value in ff.items():
    dinges.append(value)
dinges

In [96]:
df_test = pd.DataFrame.from_dict(json_dict[0])
df_test

,id,name,symbol,slug,max_supply,circulating_supply,total_supply,last_updated,quote,rank,noLazyLoad
USD,1,Bitcoin,BTC,bitcoin,21000000,18596406,18596406,2021-01-10T15:41:03.000Z,"{'price': 39790.654176302625, 'volume_24h': 65...",1,True


In [97]:
df_test2 = pd.json_normalize(json_dict[0], sep='_')
# df_test2.columns = df_test2.columns.str.strip('quote_')
df_test2

,id,name,symbol,slug,max_supply,circulating_supply,total_supply,last_updated,rank,noLazyLoad,quote_USD_price,quote_USD_volume_24h,quote_USD_percent_change_1h,quote_USD_percent_change_24h,quote_USD_percent_change_7d,quote_USD_market_cap,quote_USD_last_updated
0,1,Bitcoin,BTC,bitcoin,21000000,18596406,18596406,2021-01-10T15:41:03.000Z,1,True,39790.654176,6.573528e+10,-0.422332,-3.393856,17.79066,7.399632e+11,2021-01-10T15:41:03.000Z


In [ ]:
def 

In [66]:
df = pd.read_json(directory)
df

,props,page,query,buildId,assetPrefix,isFallback,customServer,gip,appGip
isServer,True,/beta,NaN,a976093c,https://web.archive.org/web/20210110154437/htt...,False,True,True,True
initialState,"{'app': {'locale': 'en-US', 'theme': 'DAY', 'l...",/beta,NaN,a976093c,https://web.archive.org/web/20210110154437/htt...,False,True,True,True
initialProps,"{'initialI18nStore': {'en': {'common': {}}}, '...",/beta,NaN,a976093c,https://web.archive.org/web/20210110154437/htt...,False,True,True,True
cryptocurrencyType,NaN,/beta,,a976093c,https://web.archive.org/web/20210110154437/htt...,False,True,True,True


In [65]:
import json 

with open(directory, 'r') as j:
     contents = json.loads(j.read())
        
contents
# df = pd.DataFrame.from_dict(contents, orient='index')
# df

{'props': {'isServer': True,
  'initialState': {'app': {'locale': 'en-US',
    'theme': 'DAY',
    'lang': 'en',
    'currency': {'global': 'usd', 'table': '', 'isUnconverted': False},
    'bottomBannerHeights': {},
    'browser': {},
    'window': {'width': 0, 'height': 0, 'isNarrowLayout': False},
    'modal': {'instance': 0, 'data': {}},
    'message': ''},
   'calendarEvent': {'listingLatest': {'data': []},
    'calendarEventsById': {},
    'current': {}},
   'converter': {'isConverting': False,
    'values': {'from': '',
     'to': '',
     'amount': '',
     'fromLabel': '',
     'toLabel': ''},
    'options': [],
    'conversions': {},
    'errors': {}},
   'cryptocurrency': {'listingLatest': {'page': 1,
     'sort': 'rank',
     'sortDirection': 'asc',
     'data': [{'id': 1,
       'name': 'Bitcoin',
       'symbol': 'BTC',
       'slug': 'bitcoin',
       'max_supply': 21000000,
       'circulating_supply': 18596406,
       'total_supply': 18596406,
       'last_updated': '20

In [147]:
# test_str

In [148]:
# text[-21]

In [37]:
text = test.split('\n')

for i in range(len(text)):
    if '__NEXT_DATA__' in text[i]:
        print(text[i])
#         strip_tags(text[i])

</style></head><body data-commit="a976093c" class="DAY"><div id="__next"><div class="sc-1xxpehh-1 bAVeXU"><div class="sc-1xxpehh-0 iHwvSH"><div class="xwtbyq-0 iGclcX cmc-header-desktop"><div class="sc-33i2yg-0 dOnegn"><div class="container"><div><span class="sc-12ja2s9-0 dzHJPm">Cryptocurrencies:  <a href="/web/20210110154437/https://coinmarketcap.com/all/views/all/" class="cmc-link">8,222</a></span><span class="sc-12ja2s9-0 dzHJPm">Markets:  <a href="/web/20210110154437/https://coinmarketcap.com/currencies/volume/24-hour/" class="cmc-link">34,030</a></span><span class="sc-12ja2s9-0 dzHJPm">Market Cap:  <a href="/web/20210110154437/https://coinmarketcap.com/charts/" class="cmc-link">$1,080,682,596,915</a></span><span class="sc-12ja2s9-0 dzHJPm">24h Vol:  <a href="/web/20210110154437/https://coinmarketcap.com/charts/" class="cmc-link">$164,791,539,594</a></span><span class="sc-12ja2s9-0 dzHJPm">BTC Dominance:  <a href="/web/20210110154437/https://coinmarketcap.com/charts/#dominance-per

In [ ]:
strip_tags(html)

In [30]:
import urllib, json

url = "https://web.archive.org/web/20210110154437/https://coinmarketcap.com/"
response = urllib.request.urlopen(url)
data = json.loads(response.read())
print (data)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)